In [76]:
import math

In [77]:
def get_right_hand_side(x, y, dy) -> float: # returns right hand side of the initial problem
    return 4*x*dy-(4*x*x-3)*y + pow(math.e, x*x)

def get_coefficients(h, x, y, dy): # calculates the coefficients in Runge-Kutta-Nyström formula
    f = get_right_hand_side(x, y, dy)
    k1 = (h*h/2)*f
    f = get_right_hand_side(x+h/2, y+h*dy/2+k1/4, dy+k1/h)
    k2 = (h*h/2)*f
    f = get_right_hand_side(x+h/2, y+h*dy/2+k2/4, dy+k2/h)
    k3 = (h*h/2)*f
    f = get_right_hand_side(x+h, y+h*dy+k3, dy+2*k3/h)
    k4 = (h*h/2)*f    
    return k1, k2, k3, k4

def rkn_get_end(step, y0, dy0, start_time, stop_time): # returns the function value at the end of the interval
    y = y0
    dy = dy0
    time = start_time
    while(time <= stop_time):
        k1, k2, k3, k4 = get_coefficients(step, time, y, dy)
        y = y + step*dy + (1/3)*(k1+k2+k3)
        dy = dy + (1/(3*step))*(k1+2*k2+2*k3+k4)
        time += step
    return y

def get_init_dy(alpha, beta, step, start_time, stop_time): # returns initial values of the two derivations options
    dy1 = -0.5
    dy2 = 0.5
    y1 = 0
    y2 = 0
    while((y1-beta)*(y2-beta) >= 0):
        #print("dy1=", dy1, ", dy2=", dy2)
        dy1 += dy1
        dy2 += dy2
        y1 = rkn_get_end(step, alpha, dy1, start_time, stop_time)
        y2 = rkn_get_end(step, alpha, dy2, start_time, stop_time)
        #print("y1=", y1, ", y2=", y2)
    return dy1, dy2

def rkn_shooting(epsilon, alpha, beta, step, start_time, stop_time): # returns dy sufficient for given epsilon
    dy1, dy2 = get_init_dy(alpha, beta, step, start_time, stop_time)
    y = beta + 2*epsilon # set far enough from the desired value
    
    while(abs(y-beta) > epsilon):
        dy = (dy1+dy2)/2
        #print("dy=(dy1+dy2)/2=(",dy1,"+",dy2,")/2=",dy)
        y = rkn_get_end(step, alpha, dy, start_time, stop_time)
        if(y > beta):
            dy2 = dy
            #print("y=", y, ", dy2=", dy)
        else:
            dy1 = dy   
            #print("y=", y, ", dy1=", dy)
    return dy

def rkn_solver(alpha, beta, step, epsilon, start_time, stop_time, output_path):
    y = alpha
    dy = rkn_shooting(epsilon, alpha, beta, step, start_time, stop_time)
    time = start_time
    f = open(output_path, "w") # overwrite content
    f.write("#x\t y") # first line comment
    f.write("\n")
    f.close()
    while(time <= stop_time):
        f = open(output_path, "a") # now appending
        f.write(str(round(time,3)))
        f.write("\t")
        f.write(str(round(y,3)))
        f.write("\n")
        f.close()
        k1, k2, k3, k4 = get_coefficients(step, time, y, dy)
        y = y + step*dy + (1/3)*(k1+k2+k3)
        dy = dy + (1/(3*step))*(k1+2*k2+2*k3+k4)
        time += step

In [80]:
if __name__ == "__main__":
    
    # set global variables
    alpha = -0.3
    beta = 0.2
    step = 0.01
    start_time = 0
    stop_time = 1
    epsilon = 0.01
    output_path = "output.txt"
    
    # shooting method
    rkn_solver(alpha, beta, step, epsilon, start_time, stop_time, output_path)
    